In [1]:
import pandas as pd 
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [2]:
## Load the dataset 

data = pd.read_csv('Bank_Customer_Churn_Prediction.csv')
data.head()

,customer_id,credit_score,country,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,churn
0,15634602,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,15647311,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,15619304,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,15701354,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,15737888,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
data.shape

(10000, 12)

In [4]:
## Preprocess the data 

### Drop irrelevant columns 

data = data.drop(['customer_id'], axis= 1)
data.head()

,credit_score,country,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,churn
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
## Encode cateforical variable 
label_encoder_gender = LabelEncoder()
data['gender']=label_encoder_gender.fit_transform(data['gender'])
data

,credit_score,country,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,churn
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [6]:
## onehot encode 'Geography' 

from sklearn.preprocessing import OneHotEncoder 
onehot_encoder_geo = OneHotEncoder()
geo_encoder = onehot_encoder_geo.fit_transform(data[['country']])
geo_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [7]:
onehot_encoder_geo.get_feature_names_out(['country'])

array(['country_France', 'country_Germany', 'country_Spain'], dtype=object)

In [8]:
geo_encoder.toarray()

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [9]:
geo_encoded_df = pd.DataFrame(geo_encoder.toarray(), columns= onehot_encoder_geo.get_feature_names_out(['country']))

In [10]:
geo_encoded_df

,country_France,country_Germany,country_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [11]:
## Combine one hot encoder columns with the original data 

data = pd.concat([data.drop('country', axis=1), geo_encoded_df], axis=1)
data.head()

,credit_score,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,churn,country_France,country_Germany,country_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [12]:
## Save the encoders and scaler 

with open('label_encoder_gender.pkl', 'wb') as file: 
    pickle.dump(label_encoder_gender, file)


with open('onehot_encoder_geo.pkl', 'wb') as file: 
    pickle.dump(onehot_encoder_geo, file)

In [13]:
data.head()

,credit_score,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,churn,country_France,country_Germany,country_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [14]:
## Divide the dataset into dependent and independent features 

X = data.drop('churn', axis = 1)
y = data['churn']

## Split the data into train and test sets 

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size= 0.25, random_state= 42)

## Scale the features 
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test) 

In [15]:
with open('scaler.pkl', 'wb') as file: 
    pickle.dump(scaler, file)

## ANN Implementation

In [16]:
import tensorflow as tf
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard 
import datetime 

In [17]:
(X_train.shape[1], )

(12,)

In [18]:
## Build our ANN Model 

model = Sequential([
    Dense(64, activation = 'relu', input_shape = (X_train.shape[1], )), ## HL 1 connected with i/p 
    Dense(32, activation = 'relu'), ## HL 2
    Dense(1, activation = 'sigmoid') ## O/p layer 
])

2025-04-16 09:43:50.946940: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2025-04-16 09:43:50.946985: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-04-16 09:43:50.946993: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-04-16 09:43:50.947186: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-04-16 09:43:50.947506: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [20]:
import tensorflow
opt= tensorflow.keras.optimizers.legacy.Adam(learning_rate= 0.02)

In [21]:
## compile the model 

model.compile(optimizer= opt, loss= "binary_crossentropy", metrics= ['accuracy'])

In [22]:
## Set up the tensorboard 
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard 

log_dir= "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") 
tensorflow_callback = TensorBoard(log_dir= log_dir, histogram_freq= 1)

In [23]:
## Setup Early Stopping 
early_stopping_callback = EarlyStopping(monitor='val_loss', patience= 8, restore_best_weights= True)

In [24]:
# Train the model 
history = model.fit(
    X_train, y_train, validation_data=(X_test, y_test), epochs = 100, batch_size = 32,
    callbacks = [tensorflow_callback, early_stopping_callback]
)

Epoch 1/100


2025-04-16 09:44:04.548141: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


235/235 [==============================] - 4s 13ms/step - loss: 1.2633 - accuracy: 0.7455 - val_loss: 2.6338 - val_accuracy: 0.5928
Epoch 2/100
235/235 [==============================] - 2s 8ms/step - loss: 9.4175 - accuracy: 0.7084 - val_loss: 8.8138 - val_accuracy: 0.8096
Epoch 3/100
235/235 [==============================] - 2s 8ms/step - loss: 35.0271 - accuracy: 0.7127 - val_loss: 72.9918 - val_accuracy: 0.7416
Epoch 4/100
235/235 [==============================] - 2s 8ms/step - loss: 48.8129 - accuracy: 0.7228 - val_loss: 40.7172 - val_accuracy: 0.8064
Epoch 5/100
235/235 [==============================] - 2s 8ms/step - loss: 59.6945 - accuracy: 0.7333 - val_loss: 14.4744 - val_accuracy: 0.8192
Epoch 6/100
235/235 [==============================] - 2s 8ms/step - loss: 71.4697 - accuracy: 0.7152 - val_loss: 151.4600 - val_accuracy: 0.5776
Epoch 7/100
235/235 [==============================] - 2s 8ms/step - loss: 100.8880 - accuracy: 0.7285 - val_loss: 182.0368 - val_accuracy: 0.58

In [25]:
model.save('model.h5')

/Users/saivineethgrandhi/Documents/C/GenAI/SimpleANNImplementation/venv/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [26]:
## Load TensorBoard extension 

%load_ext tensorboard 

In [28]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 58309), started 0:00:07 ago. (Use '!kill 58309' to kill it.)

In [29]:
## Kill the tensorboard

!kill 58309